**Facial Landmarks Number Order**

* JAW_POINTS = 0 -17
* RIGHT_BROW_POINTS = 17 - 21
* LEFT_BROW = 22-27
* NOSE_POINTS = 27 - 35
* RIGHT_EYE = 36-42
* LEFT_EYE = 42 - 38
* MOUTH_POINTS = 48-61

In [11]:
import cv2
import dlib
import numpy as np

predictor = dlib.shape_predictor("D:/Computer_Vision_Recap/shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

class TooManyFaces(Exception):
    pass

class NoFaces(Exception):
    pass


def get_facial_landmarks(image):
    # func gets the facial landmark from image
    rects = detector(image, 1)          # gets ROI of the facial landmark
    
    if(len(rects) > 1):
        raise TooManyFaces
    if(len(rects)) == 0:
        raise  NoFaces
        
    # returns reformatted array for annot landmarks func
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

def annotate_landmarks(image, landmarks):
    # func, numbers the detected landmarks
    image = image.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0,0], point[0,1])
        cv2.putText(image, str(idx), pos,
                   cv2.FONT_HERSHEY_DUPLEX, 0.4, (255, 0, 0), 1)
        cv2.circle(image, pos, 3, (0,0, 255), 1)
    return image

image = cv2.imread("D:/Computer_Vision_Recap/images/Trump.jpg")
landmarks = get_facial_landmarks(image)
image_with_landmarks = annotate_landmarks(image, landmarks)

cv2.imshow("Detected landmarks", image_with_landmarks)
cv2.waitKey(0)
cv2.destroyAllWindows()

***
#### Face Swapper

In [4]:
# not fully understood and working

import cv2
import dlib
import numpy as np


predictor_path = "D:/Computer_Vision_Recap/shape_predictor_68_face_landmarks.dat"
scale_factor = 1
feather_amount = 11

# facial landmarks
face_points = list(range(17, 68))
mouth_points = list(range(48, 61))
right_brow_points = list(range(17, 22))
left_brow_points = list(range(22, 27))
right_eye_points = list(range(36, 42))
left_eye_points = list(range(42, 48))
nose_points = list(range(27, 35))
jaw_points = list(range(0, 17))


# points to line up im1 and im2
allign_points = (left_brow_points + right_brow_points + left_eye_points + 
             right_eye_points + nose_points + mouth_points)

# points from im2 to be overlaid on im1
overlay_points = [
    left_eye_points + right_eye_points + left_brow_points + right_brow_points + 
    nose_points + mouth_points
]


color_correct_blur_frac = 0.6            # amount of blut to use during color correction

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)


def get_facial_landmarks(image):
    # func gets the facial landmark from image
    rects = detector(image, 1)          # gets ROI of the facial landmark
    
    if(len(rects) > 1):
        raise TooManyFaces
    if(len(rects)) == 0:
        raise  NoFaces
        
    # returns reformatted array for annot landmarks func
    return np.matrix([[p.x, p.y] for p in predictor(image, rects[0]).parts()])

def annotate_landmarks(image, landmarks):
    # func, numbers the detected landmarks
    image = image.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0,0], point[0,1])
        cv2.putText(image, str(idx), pos,
                   cv2.FONT_HERSHEY_DUPLEX, 0.4, (255, 0, 0), 1)
        cv2.circle(image, pos, 3, (0,0, 255), 1)
    return image

def draw_convex_hull(im, points, color):
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color = color)
    
def get_face_mask(im, landmarks):
    im = np.zeros(im.shape[:2], dtype = np.float64)
    
    for group in overlay_points:
        draw_convex_hull(im, landmarks[group], color = 1)
        
    im = np.array([im, im, im]).transpose((1, 2, 0))
    
    im = (cv2.GaussianBlur(im, (feather_amount, feather_amount), 0) > 0) * 1.0
    im = cv2.GaussianBlur(im, (feather_amount, feather_amount), 0)
    return im


def transformation_from_points(points1, points2):
    """
    ret affine transformation matrix for warping op of image 1 and image 2
    using procustes method
    """
    points1 = points1.astype(np.float64)
    points2 = points2.astype(np.float64)

    c1 = np.mean(points1, axis = 0)
    c2 = np.mean(points2, axis = 0)
    points1 -= c1
    points2 -= c2
    
    s1 = np.std(points1)
    s2 = np.std(points2)
    points1 /= s1
    points2 /= s2
    
    U, S, Vt = np.linalg.svd(points1.T * points2)
    R = (U * Vt).T
    
    return np.vstack([
        np.hstack(((s2 / s1) * R,
                  c2.T - (s2 / s1) * R * c1.T)),
        np.matrix([0., 0., 1.])
    ])

def read_im_and_landmarks(image):
    """
    func reads and resizes image for faster computationa and gets landmarks
    returns image along with its detected landmarks
    """
    im = image
    im = cv2.resize(im, None, fx = 1, fy = 1, interpolation = cv2.INTER_LINEAR)
    im = cv2.resize(im, (im.shape[1] * scale_factor, im.shape[0] * scale_factor))
    
    s = get_facial_landmarks(im)
    return im, s

def warp_im(image, M, dshape):
    output_im = np.zeros(dshape, dtype=image.dtype)
    # does the warp opp and the borders is made transparent and ret the result
    cv2.warpAffine(image,
                  M[:2],
                  (dshape[1], dshape[0]),
                  dst = output_im,
                  borderMode = cv2.BORDER_TRANSPARENT,
                  flags = cv2.WARP_INVERSE_MAP)
    return output_im

def correct_colours(im1, im2, landmarks1):
    # does the color correction & blurring operations
    blur_amount = color_correct_blur_frac * np.linalg.norm(
            np.mean(landmarks1[left_eye_points], axis = 0) - 
            np.mean(landmarks1[right_eye_points], axis = 0))
    blur_amount = int(blur_amount)
    if blur_amount % 2 == 0:
        blur_amount += 1
    im1_blur = cv2.GaussianBlur(im1, (blur_amount, blur_amount), 0)
    im2_blur = cv2.GaussianBlur(im2, (blur_amount, blur_amount), 0)
    
    # avoids zero div Xception
    im2_blur += (128 * (im2_blur <= 1.0)).astype(im2_blur.dtype)
#     im2_blur += 128 * (im2_blur <= 1.0)
    
    return (im2.astype(np.float64) * im1_blur.astype(np.float64) / 
            im2_blur.astype(np.float64))
    
    
def swappy(image1, image2):
    """
    uses above functions here and swaps the images
    """
    im1, landmarks1 = read_im_and_landmarks(image1)
    im2, landmarks2 = read_im_and_landmarks(image2)
    
    M = transformation_from_points(landmarks1[allign_points], landmarks2[allign_points])
    
    mask = get_face_mask(im2, landmarks2)
    warped_mask = warp_im(mask, M, im1.shape)
    combined_mask = np.max([get_face_mask(im1, landmarks1), warped_mask],
                          axis = 0)
    
    warped_im2 = warp_im(im2, M, im1.shape)
    warped_color_corrected_im2 = correct_colours(im1, warped_im2, landmarks1)
    
    output_im = im1 * (1.0 - combined_mask) + warped_color_corrected_im2 * combined_mask
    cv2.imwrite("Swapped_face.jpg", output_im)
    output = cv2.imread("Swapped_face.jpg")
    cv2.imshow("The output", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



image1 = cv2.imread("D:/Computer_Vision_Recap/images/Trump.jpg")    # Base image
image2 = cv2.imread("D:/Computer_Vision_Recap/images/Hillary.jpg")  # image to be mask on top base image

swapped_faces_image = swappy(image1,image2)